## Import packages

In [11]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

## Data preprocessing

In [12]:
with open('../datasets/data.json') as file:
    data = json.load(file)

In [13]:
training_sentences = []
training_labels = []
labels = []
responses = []

In [14]:
for intent in data['intents']:
    for pattern in intent['questions']:
        training_sentences.append(pattern)
        training_labels.append(intent['intent'])
    responses.append(intent['responses'])
    
    if intent['intent'] not in labels:
        labels.append(intent['intent'])
        
num_classes = len(labels)


## Saving intents in json

In [15]:
labels_json ={ 
    "intent_labels": labels  
} 
    
out_file = open("../datasets/labels.json", "w") 
    
json.dump(labels_json, out_file, indent = 6) 
    
out_file.close() 

In [16]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [17]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)

In [18]:
X = tf.keras.preprocessing.sequence.pad_sequences(sequences, truncating='post', maxlen=max_len)
Y = np.array(training_labels)

## Model Building

In [19]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-06-07 06:28:37.708087: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-07 06:28:37.708119: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-07 06:28:37.708143: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (surajkarki-inspiron5567): /proc/driver/nvidia/version does not exist
2022-06-07 06:28:37.708409: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 37)                629       
                                                                 
Total params: 17,173
Trainable params: 17,173
Non-trainable params: 0
____________________________________________________

## Train the model

In [21]:
epochs = 500
history = model.fit(X, Y, epochs=epochs)

Epoch 1/500
14/14 [==============================] - 1s 4ms/step - loss: 3.6084 - accuracy: 0.0252
Epoch 2/500
14/14 [==============================] - 0s 5ms/step - loss: 3.6017 - accuracy: 0.0275
Epoch 3/500
14/14 [==============================] - 0s 5ms/step - loss: 3.5938 - accuracy: 0.0458
Epoch 4/500
14/14 [==============================] - 0s 5ms/step - loss: 3.5840 - accuracy: 0.0526
Epoch 5/500
14/14 [==============================] - 0s 7ms/step - loss: 3.5717 - accuracy: 0.0458
Epoch 6/500
14/14 [==============================] - 0s 6ms/step - loss: 3.5560 - accuracy: 0.0458
Epoch 7/500
14/14 [==============================] - 0s 8ms/step - loss: 3.5371 - accuracy: 0.0915
Epoch 8/500
14/14 [==============================] - 0s 7ms/step - loss: 3.5155 - accuracy: 0.0847
Epoch 9/500
14/14 [==============================] - 0s 3ms/step - loss: 3.4944 - accuracy: 0.0778
Epoch 10/500
14/14 [==============================] - 0s 5ms/step - loss: 3.4745 - accuracy: 0.0778
Epoch 11/

In [24]:
model.save("../lucy_models/lucy_simple_ann")

INFO:tensorflow:Assets written to: ../lucy_models/lucy_simple_ann/assets


In [34]:
input_text = "hello"
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(input_text)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(input_text)

In [35]:
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, truncating='post', maxlen=max_len)

In [36]:
y_pred = model.predict(np.expand_dims(padded_sequences[0], axis=0))[0]

1/1 [==============================] - 0s 31ms/step


In [37]:
print(np.argmax(y_pred))

18
